In [1]:
import os
import subprocess
import logging
from pathlib import Path
from tqdm import tqdm
import sys
from google.colab import drive
import ipywidgets as widgets
from IPython.display import display, HTML
import tempfile
import shutil

class VideoProcessor:
    def __init__(self):
        self.setup_logging()
        self.temp_dir = tempfile.mkdtemp()
        self.input_folder = '/content/drive/MyDrive/Uploader Video'
        self.output_folder = '/content/drive/MyDrive/output folder'
        self.watermark_path = '/content/drive/MyDrive/c3.png'
        self.config = {
            'speed_factor': 1.0,
            'pitch_factor': 1.0,
            'brightness': 0.0,
            'contrast': 1.0,
            'saturation': 1.0,
            'watermark_size': 50,
            'crop_percentage': 1.0,
            'volume_boost': 0,
            'flip_video': False,
            'flip_watermark': False,
            'watermark_position': 'top-right'
        }

    def setup_logging(self):
        logging.basicConfig(
            level=logging.DEBUG,
            format='%(asctime)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler('video_processing.log'),
                logging.StreamHandler()
            ]
        )

    def get_watermark_position(self):
        positions = {
            'top-right': 'W-w-10:10',
            'top-left': '10:10',
            'bottom-right': 'W-w-10:H-h-10',
            'bottom-left': '10:H-h-10'
        }
        return positions.get(self.config['watermark_position'], 'W-w-10:10')

    def create_complex_filter(self):
        filters = []

        if self.config['speed_factor'] != 1.0:
            filters.append(f'setpts=1/{self.config["speed_factor"]}*PTS')

        if self.config['flip_video']:
            filters.append('hflip')

        filters.extend([
            'scale=trunc(iw/2)*2:trunc(ih/2)*2',
            f'eq=brightness={self.config["brightness"]}:'
            f'contrast={self.config["contrast"]}:'
            f'saturation={self.config["saturation"]}'
        ])

        main_video = '[0:v]' + ','.join(filters) + '[v]'
        watermark = f'[1:v]scale={self.config["watermark_size"]}:-1[vw]'

        if self.config['flip_watermark']:
            watermark += ',hflip'

        overlay = f'[v][vw]overlay={self.get_watermark_position()}[vout]'

        return main_video + ';' + watermark + ';' + overlay

    def create_audio_filter(self):
        filters = []

        if self.config['speed_factor'] != 1.0 or self.config['pitch_factor'] != 1.0:
            speed = self.config['speed_factor']
            pitch = self.config['pitch_factor']
            filters.extend([
                f'asetrate=44100*{speed}',
                f'aresample=44100',
                f'rubberband=pitch={pitch}'
            ])

        if self.config['volume_boost'] != 0:
            filters.append(f'volume={self.config["volume_boost"]}dB')

        return ','.join(filters) if filters else 'anull'

    def process_video(self, input_file, watermark_path, output_file):
        try:
            input_file = str(Path(input_file))
            watermark_path = str(Path(watermark_path))
            output_file = str(Path(output_file))

            command = [
                'ffmpeg',
                '-i', input_file,
                '-i', watermark_path,
                '-filter_complex', self.create_complex_filter(),
                '-af', self.create_audio_filter(),
                '-map', '[vout]',
                '-map', '0:a',
                '-map_metadata', '-1',
                '-c:v', 'h264_nvenc',
                '-preset', 'slow',
                '-c:a', 'aac',
                '-y', output_file
            ]

            logging.debug(f"FFmpeg command: {' '.join(command)}")

            process = subprocess.Popen(
                command,
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE,
                universal_newlines=True
            )

            # Capture stdout and stderr in real-time
            for line in process.stderr:
                print(line.strip())

            process.communicate()

            if process.returncode != 0:
                logging.error(f"Error processing video: {input_file}")
                raise subprocess.CalledProcessError(process.returncode, command)
            logging.info(f"Successfully processed {input_file}")
            return True

        except Exception as e:
            logging.error(f"Error processing {input_file}: {str(e)}")
            return False

    def create_ui_controls(self):
        return {
            'input_folder': widgets.Text(
                value=self.input_folder,
                description='Input Folder:',
                style={'description_width': 'initial'}
            ),
            'output_folder': widgets.Text(
                value=self.output_folder,
                description='Output Folder:',
                style={'description_width': 'initial'}
            ),
            'watermark_path': widgets.Text(
                value=self.watermark_path,
                description='Watermark Path:',
                style={'description_width': 'initial'}
            ),
            'speed': widgets.FloatSlider(
                value=1.1,
                min=0.5,
                max=2.0,
                step=0.1,
                description='Speed:'
            ),
            'pitch': widgets.FloatSlider(
                value=1.1,
                min=0.5,
                max=2.0,
                step=0.1,
                description='Pitch:'
            ),
            'brightness': widgets.FloatSlider(
                value=0.2,
                min=-1.0,
                max=1.0,
                step=0.1,
                description='Brightness:'
            ),
            'contrast': widgets.FloatSlider(
                value=1.2,
                min=0.0,
                max=2.0,
                step=0.1,
                description='Contrast:'
            ),
            'saturation': widgets.FloatSlider(
                value=1.3,
                min=0.0,
                max=2.0,
                step=0.1,
                description='Saturation:'
            ),
            'volume': widgets.FloatSlider(
                value=1.0,
                min=-20.0,
                max=20.0,
                step=1.0,
                description='Volume (dB):'
            ),
            'flip_video': widgets.Checkbox(
                value=True,
                description='Flip Video'
            ),
            'flip_watermark': widgets.Checkbox(
                value=False,
                description='Flip Watermark'
            ),
            'watermark_position': widgets.Dropdown(
                options=['top-right', 'top-left', 'bottom-right', 'bottom-left'],
                value='top-right',
                description='Watermark Position:'
            )
        }

    def process_videos(self, controls, progress_bar):
        self.input_folder = controls['input_folder'].value
        self.output_folder = controls['output_folder'].value
        self.watermark_path = controls['watermark_path'].value

        self.config.update({
            'speed_factor': controls['speed'].value,
            'pitch_factor': controls['pitch'].value,
            'brightness': controls['brightness'].value,
            'contrast': controls['contrast'].value,
            'saturation': controls['saturation'].value,
            'volume_boost': controls['volume'].value,
            'flip_video': controls['flip_video'].value,
            'flip_watermark': controls['flip_watermark'].value,
            'watermark_position': controls['watermark_position'].value
        })

        if not os.path.exists(self.input_folder):
            print(f"Error: Input folder {self.input_folder} does not exist!")
            return

        if not os.path.exists(self.watermark_path):
            print(f"Error: Watermark file {self.watermark_path} does not exist!")
            return

        os.makedirs(self.output_folder, exist_ok=True)

        video_extensions = ('.mp4', '.avi', '.mov', '.mkv')
        videos = [f for f in os.listdir(self.input_folder)
                 if f.lower().endswith(video_extensions)]

        if not videos:
            print("No video files found in the input folder!")
            return

        progress_bar.max = len(videos)
        print(f"Processing {len(videos)} videos...")

        for idx, video_name in enumerate(tqdm(videos)):
            input_path = os.path.join(self.input_folder, video_name)
            output_path = os.path.join(self.output_folder, f'processed_{video_name}')

            logging.info(f"Processing video: {input_path}")
            logging.info(f"Output: {output_path}")
            logging.info(f"Watermark: {self.watermark_path}")

            success = self.process_video(input_path, self.watermark_path, output_path)

            if success:
                progress_bar.value = idx + 1
            else:
                print(f"Failed to process {video_name}")

        print("Processing complete! Check the log file for details.")

def main():
    print("Mounting Google Drive...")
    drive.mount('/content/drive')

    processor = VideoProcessor()
    controls = processor.create_ui_controls()

    print("\nConfigure processing settings:")
    for control in controls.values():
        display(control)

    process_button = widgets.Button(description="Process Videos")
    progress_bar = widgets.IntProgress(value=0, min=0, max=1, description="Processing:")

    def on_button_click(b):
        process_button.disabled = True  # Disable button during processing
        processor.process_videos(controls, progress_bar)
        process_button.disabled = False  # Enable button after processing

    process_button.on_click(on_button_click)
    display(process_button, progress_bar)

if __name__ == "__main__":
    main()


Mounting Google Drive...
Mounted at /content/drive

Configure processing settings:


Text(value='/content/drive/MyDrive/Uploader Video', description='Input Folder:', style=DescriptionStyle(descri…

Text(value='/content/drive/MyDrive/output folder', description='Output Folder:', style=DescriptionStyle(descri…

Text(value='/content/drive/MyDrive/c3.png', description='Watermark Path:', style=DescriptionStyle(description_…

FloatSlider(value=1.1, description='Speed:', max=2.0, min=0.5)

...

frame=  675 fps=179 q=32.0 size=    5376kB time=00:00:22.27 bitrate=1977.5kbits/s dup=0 drop=133 speed=5.92x
frame=  766 fps=176 q=31.0 Lsize=    6810kB time=00:00:25.75 bitrate=2166.2kbits/s dup=0 drop=151 speed=5.91x
video:6376kB audio:409kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.375361%
[aac @ 0x58cb13bc0b40] Qavg: 787.980
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-

 93%|█████████▎| 13/14 [15:11<00:42, 42.69s/it]

frame=  493 fps=177 q=24.0 size=    4352kB time=00:00:16.23 bitrate=2196.2kbits/s dup=0 drop=96 speed=5.83x
frame=  522 fps=172 q=23.0 Lsize=    4914kB time=00:00:17.34 bitrate=2320.6kbits/s dup=0 drop=102 speed=5.73x
video:4626kB audio:271kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.363162%
[aac @ 0x5c26c77749c0] Qavg: 197.086
ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-l

100%|██████████| 14/14 [15:16<00:00, 65.43s/it]

frame=  469 fps=144 q=28.0 size=    3584kB time=00:00:15.39 bitrate=1906.8kbits/s dup=0 drop=92 speed=4.73x
frame=  485 fps=139 q=22.0 Lsize=    4173kB time=00:00:16.11 bitrate=2121.2kbits/s dup=0 drop=95 speed=4.62x
video:3904kB audio:252kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.404155%
[aac @ 0x588042dc4280] Qavg: 190.505
Processing complete! Check the log file for details.
